In [37]:
feature_number = 20
test_size = 0.9
y_feature = 'resale_price'
token = '84e47768480771b0f65f16b3f25447e5ab6b88d174cccb8271d1d0a3e4fdf3ce875e7f9f2612ad64f6f107644f49d8c6d9cd2619469de103753b4ae7556e703a'
backendSever = 'ibm_lagos'


In [38]:
import time
import pandas as pd
import numpy as np
import qiskit
import matplotlib.pyplot as plt
from qiskit_algorithms.optimizers import L_BFGS_B
from qiskit_machine_learning.algorithms.regressors import NeuralNetworkRegressor, VQR
from qiskit.circuit.library import RealAmplitudes, ZZFeatureMap
from qiskit_machine_learning.circuit.library import QNNCircuit
from sklearn.model_selection import train_test_split

In [39]:
data = pd.read_csv(f'../data/flat_data/{feature_number}F/data.csv')
X = data.drop(columns=y_feature)
y = data[y_feature]

In [40]:
train_data, test_data, train_labels , test_labels = train_test_split(X, y, test_size=test_size, random_state=101)

In [41]:
def callback_graph(weights, obj_func_eval):
    clear_output(wait=True)
    objective_func_vals.append(obj_func_eval)
    plt.title("Objective function value against iteration")
    plt.xlabel("Iteration")
    plt.ylabel("Objective function value")
    plt.plot(range(len(objective_func_vals)), objective_func_vals)
    plt.show()

In [42]:
from qiskit.primitives import Estimator

In [43]:
from qiskit_ibm_runtime import Estimator, QiskitRuntimeService

service = QiskitRuntimeService(
     channel='ibm_quantum',
    instance='ibm-q/open/main',
    token=token
)
backend = service.backend(backendSever)
estimator = Estimator(backend=backend)

In [44]:
feature_map = ZZFeatureMap(20,reps=1,insert_barriers='True')
ansatz = RealAmplitudes(20,reps=1)

In [45]:
vqr = VQR(
    feature_map=feature_map,
    ansatz=ansatz,
    optimizer=L_BFGS_B(maxiter=5))

In [46]:
objective_func_vals = []

start_time = time.time()
vqr.fit(train_data,train_labels)
elapsed_time = time.time() - start_time

print(f'Time spent to fit: {elapsed_time}' )

KeyboardInterrupt: 

In [ ]:
start_time = time.time()
predicted_vqr_test_labels = vqr.predict(test_data)
elapsed_time = time.time() - start_time
print(f'Time spent to predict: {elapsed_time}' )

In [ ]:
start_time = time.time()
score = vqr.score(train_data,train_labels)
elapsed_time = time.time() - start_time
print(f'score : {score} \ntime to score: {elapsed_time}' )

In [ ]:
from sklearn import metrics

print('MAE:', metrics.mean_absolute_error(test_labels, predicted_vqr_test_labels))
print('MSE:', metrics.mean_squared_error(test_labels, predicted_vqr_test_labels))
print('RMSE:', np.sqrt(metrics.mean_squared_error(test_labels, predicted_vqr_test_labels)))